In [ ]:
 pip install pulp

     |████████████████████████████████| 40.6MB 105kB/s 


In [ ]:
import pandas as pd
import numpy as np

import pulp as plp

In [ ]:
url = 'https://github.com/tipsytc/RevenueManagement/blob/main/students.csv?raw=true'
df = pd.read_csv(url, index_col=0)
df.head(3)


,BEER,NONALCOHOL,BURGER,HOTDOG,TW1,TW2,TW3
0,3,0,1,2,0,0,1
1,4,4,2,2,0,1,1
2,4,3,3,1,1,0,1


## Baseline Revenue and Attendance
Suppose the organizer has no prior knowledge of students' preferred time window. Revenue and attendance can be easily calculated. For the ease of calculation, we assume the students complete their intended purchases at the first time window they are admitted. The revenue and attendance will be referred as the baseline of our optimization model.

In [ ]:
item_list = ["BEER", "NONALCOHOL", "BURGER", "HOTDOG"]

In [ ]:
#We bootstrap a sample of size of 1500
#since the theoretical max attendance is 1500

sample = df.iloc[np.random.randint(1500, size=1500)].reset_index(drop = True)

#add an ID column to keep track
sample["STUDENT_ID"] = np.array([i+1 for i in range(sample.shape[0])])

#Binary indicator whether student is admitted in Time Window i
sample["TW1AD"] = 0
sample["TW2AD"] = 0
sample["TW3AD"] = 0

In [ ]:
sample_tw1 = sample[sample.TW1 == 1].reset_index(drop = True)
# sample_tw1.head(3)

In [ ]:
capacity = 500
item_ctr_dict = {"BEER":0, "NONALCOHOL":0, "BURGER":0, "HOTDOG":0}

rev = 0
admit_list = []

In [ ]:
#Time Window 1

tw1_ctr = 0

for i in range(sample_tw1.shape[0]):
  student = sample_tw1.iloc[i]
  if capacity > 0:
    capacity -= 1
    tw1_ctr += 1
    admit_list.append(student["STUDENT_ID"])
    sample.loc[sample["STUDENT_ID"] == student["STUDENT_ID"],"TW1AD"] = 1
    sample.loc[sample["STUDENT_ID"] == student["STUDENT_ID"],"TW2AD"] = student["TW2"]
    for item in item_list:
      item_ctr_dict[item] += student[item]
      student[item] = 0

#students admitted in TW1 and intended to stay in TW2 will stay
sample.loc[(sample["TW1AD"] == 1) & (sample["TW2"] == 1), "TW2AD"] = 1
carryover = sum(sample_tw1["TW2"])

print("Dring 12:00 - 3:00 (TW1): " + str(sum(sample["TW1AD"])) + " were admitted, " + str(sample_tw1.shape[0] - tw1_ctr) + " were not admitted due to capacity limit.")
print(str(carryover) + " of them are intended to stay during the next time window.")
print("Total sales:")
print(item_ctr_dict)

Dring 12:00 - 3:00 (TW1): 500 were admitted, 41 were not admitted due to capacity limit.
244 of them are intended to stay during the next time window.
Total sales:
{'BEER': 1192, 'NONALCOHOL': 1345, 'BURGER': 508, 'HOTDOG': 571}


In [ ]:
#sample_tw2 excludes the students already admitted in TW1
sample_tw2 = sample[(sample.TW2 == 1) & (sample.TW1AD == 0)].reset_index(drop = True)
sample_tw2.shape

(504, 11)

In [ ]:
#we update the capacity by subtracting the carryover capacity
capacity = 500 - carryover
print("Available capacity at the beginning of TW2: " + str(capacity))

tw2_ctr = 0
for i in range(sample_tw2.shape[0]):
  student = sample_tw2.iloc[i]
  if capacity > 0:
    capacity -= 1
    tw2_ctr += 1
    admit_list.append(student["STUDENT_ID"])
    sample.loc[sample["STUDENT_ID"] == student["STUDENT_ID"],"TW2AD"] = 1
    for item in item_list:
      item_ctr_dict[item] += student[item]
      student[item] = 0

#students admitted in TW2 and intended to stay in TW3 will stay
sample.loc[(sample["TW2AD"] == 1) & (sample["TW3"] == 1), "TW3AD"] = 1
carryover = sum(sample[sample["TW2AD"] == 1]["TW3"])

print("Dring 3:00 - 6:00 (TW2): " + str(tw2_ctr) + " were admitted, " + str(sample_tw2.shape[0] - tw2_ctr) + " were not admitted due to capacity limit.")
print(str(carryover) + " of them are intended to stay during the next time window.")
print("Total sales:")
print(item_ctr_dict)

Available capacity at the beginning of TW2: 256
Dring 3:00 - 6:00 (TW2): 256 were admitted, 248 were not admitted due to capacity limit.
365 of them are intended to stay during the next time window.
Total sales:
{'BEER': 1804, 'NONALCOHOL': 2021, 'BURGER': 788, 'HOTDOG': 864}


In [ ]:
#sample_tw3 excludes the students already admitted in TW1&TW2
sample_tw3 = sample[(sample.TW3 == 1) & (sample.TW2AD == 0)].reset_index(drop = True)
sample_tw3.shape

(874, 11)

In [ ]:
#we update the capacity by subtracting the carryover capacity
capacity = 500 - carryover
print("Available capacity at the beginning of TW3: " + str(capacity))

tw3_ctr = 0
for i in range(sample_tw3.shape[0]):
  student = sample_tw3.iloc[i]
  if capacity > 0:
    capacity -= 1
    tw3_ctr += 1
    admit_list.append(student["STUDENT_ID"])
    sample.loc[sample["STUDENT_ID"] == student["STUDENT_ID"],"TW3AD"] = 1
    for item in item_list:
      item_ctr_dict[item] += student[item]
      student[item] = 0

print("Dring 3:00 - 6:00 (TW2): " + str(tw3_ctr) + " were admitted, " + str(sample_tw3.shape[0] - tw3_ctr) + " were not admitted due to capacity limit.")
print("Total sales:")
print(item_ctr_dict)

Available capacity at the beginning of TW3: 135
Dring 3:00 - 6:00 (TW2): 135 were admitted, 739 were not admitted due to capacity limit.
Total sales:
{'BEER': 2151, 'NONALCOHOL': 2395, 'BURGER': 922, 'HOTDOG': 1005}


In [ ]:
total_rev = (item_ctr_dict["BEER"] + item_ctr_dict["BURGER"])*3.5 + (item_ctr_dict["NONALCOHOL"] + item_ctr_dict["HOTDOG"])*1.5

sample["REVENUE"] = (sample["BEER"] + sample["BURGER"])*3.5 + (sample["NONALCOHOL"] + sample["HOTDOG"])*1.5


sample["MODIFIED_REV"] = sample["REVENUE"] * (sample["TW1AD"] + sample["TW2AD"] + sample["TW3AD"]) / (sample["TW1"] + sample["TW2"] + sample["TW3"])
modified_rev = sum(sample["MODIFIED_REV"])
print("Total visits: " + str(sum(sample["TW1AD"]) + sum(sample["TW2AD"]) + sum(sample["TW3AD"])))
print("Total students admitted: " + str(len(admit_list)))
print("Total revenue: " + str(total_rev))

#Assume students make their purchase equally distributed to the time windows
#they are admitted to.
print("Modified total revenue: " + str(modified_rev))


Total visits: 1483
Total students admitted: 891
Total revenue: 15855.5
Modified total revenue: 13576.75


### Model 1: Revenue Maximization

Now we asssume we have the prior knowledge of their intended time windows, and aim to optimize the total revenue (modified).


Notation:
- $y_i$: binary variable whether student $i$ is admitted to the event
- $x_{it}$: binary variable whether student $i$ is admitted to the event at time window $t$
- $c_t$: number of students admitted at time window $t$

- $p_{j}$: price of item $j$
- $d_{ij}$: student $i$'s demand of item $j$

- $C$: capacity (500)
- $t$: time window index
- $n$: number of interested students

Objective:
$$\max  \sum_{i} y_i ADJ\_REV_i$$

Constraints:
\begin{align*} 
y_i & > \frac{1}{M}\sum_{t} x_{it} \hspace{1cm} i \in \{1, \dots, n\} \\
c_t & \le C\\
\end{align*}

In [ ]:
T = range(3)
I = range(1500)
M = 1996
C = 500

tw_dict = {0: np.array(sample["TW1"]),
           1: np.array(sample["TW2"]),
           2: np.array(sample["TW3"])}

In [ ]:
# Define problem
model1 = plp.LpProblem("Revenue_Maximization", plp.LpMaximize)

In [ ]:
X = plp.LpVariable.dicts(name='X', indexs=(I,T), cat=plp.LpBinary)
Y = plp.LpVariable.dicts(name='Y', indexs=(I), cat=plp.LpBinary)
Z = plp.LpVariable.dicts(name='Z', indexs=(I))

In [ ]:
#Maximize total revenue

model1 += plp.lpSum(Z[i] * sample.iloc[i]["REVENUE"] for i in I)

In [ ]:
for i in I:
  model1 += M*Y[i] >= plp.lpSum(X[i][j] for j in T)
  model1 += Y[i] <= plp.lpSum(X[i][j] for j in T)
  model1 += Z[i] == plp.lpSum(X[i][j] for j in T) / sum([tw_dict[j][i] for j in range(3)])
  for j in T:
    #a student is admitted at a specific time window if only he/she intends so
    model1 += X[i][j] <= tw_dict[j][i]

#we cannot force admitted students to leave
#if they intend to stay for the next time window

for i in I:
  model1 += X[i][1] >= X[i][0]*tw_dict[1][i]
  model1 += X[i][2] >= X[i][1]*tw_dict[2][i]


for j in T:
  model1 += plp.lpSum(X[i][j] for i in I) <= C

In [ ]:
model1.solve()

1

In [ ]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[model1.status])

# Results

total_visit = 0
for i in range(1500):
  for j in range(3):
    total_visit += X[i][j].varValue
print("Total visits: " + str(total_visit))

total_student_admits = 0
for i in range(1500):
  total_student_admits += Y[i].varValue
print("Total students admitted: " + str(total_student_admits))

obj = plp.value(model1.objective)
print("Total revenue: " + str(obj))

Solution Status: Optimal
Total visits: 1393.0
Total students admitted: 977.0
Total revenue: 17804.75


In [ ]:
#For debug only
# for i in range(50):
#   print(sample.iloc[i][["TW1","TW2","TW3"]])
#   print(X[i][0].varValue, X[i][1].varValue,X[i][2].varValue,Y[i].varValue)

### Model 2: Attendance Maximization
Now we asssume we have the prior knowledge of their intended time windows, and aim to admit as many different students as possible.

Notation:
- $y_i$: binary variable whether student $i$ is admitted to the event
- $x_{it}$: binary variable whether student $i$ is admitted to the event at time window $t$
- $c_t$: number of students admitted at time window $t$
- $C$: capacity (500)
- $t$: time window index
- $n$: number of interested students

Objective:
$$\max \sum_i y_i$$

Constraints:
\begin{align*} 
y_i & > \frac{1}{M}\sum_{t} x_{it} \hspace{1cm} i \in \{1, \dots, n\}\\
c_t & \le C\\
\end{align*}

In [ ]:
# Define problem
model2 = plp.LpProblem("Attendance Maximization", plp.LpMaximize)

X = plp.LpVariable.dicts(name='X', indexs=(I,T), cat=plp.LpBinary)
Y = plp.LpVariable.dicts(name='Y', indexs=(I), cat=plp.LpBinary)
Z = plp.LpVariable.dicts(name='Z', indexs=(I))

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
#Maximize total number of students admitted

model2 += plp.lpSum(Y[i] for i in I)

In [ ]:
for i in I:
  model2 += M*Y[i] >= plp.lpSum(X[i][j] for j in T)
  model2 += Y[i] <= plp.lpSum(X[i][j] for j in T)
  model2 += Z[i] == plp.lpSum(X[i][j] for j in T) / sum([tw_dict[j][i] for j in range(3)])

  for j in T:
    model2 += X[i][j] <= tw_dict[j][i]

#we cannot force admitted students to leave
#if they intend to stay for the next time window

for i in I:
  model2 += X[i][1] >= X[i][0]*tw_dict[1][i]
  model2 += X[i][2] >= X[i][1]*tw_dict[2][i]

for j in T:
  model2 += plp.lpSum(X[i][j] for i in I) <= C

In [ ]:
model2.solve()

1

In [ ]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[model2.status])

total_visit = 0
for i in range(1500):
  for j in range(3):
    total_visit += X[i][j].varValue
print("Total visits: " + str(total_visit))

total_student_admits = 0
for i in range(1500):
  total_student_admits += Y[i].varValue
print("Total students admitted: " + str(total_student_admits))

rev = 0
for i in range(1500):
  rev += Z[i].varValue * sample.iloc[i]["REVENUE"]
print("Total revenue: " + str(rev))

Solution Status: Optimal
Total visits: 979.0
Total students admitted: 979.0
Total revenue: 14288.333332614997


### We can see this objective forces each student to stay only for 1 time window.

### Model 3: Multi-objective
Now we asssume we have the prior knowledge of their intended time windows, and aim to admit as many different students as possible, while maintaining 90% of the optimal revenue ($\epsilon$) in Model 2.

Notation:
- $y_i$: binary variable whether student $i$ is admitted to the event
- $x_{it}$: binary variable whether student $i$ is admitted to the event at time window $t$
- $c_t$: number of students admitted at time window $t$
- $C$: capacity (500)
- $t$: time window index
- $n$: number of interested students

Objective:
$$\max \sum_{i} y_i $$

Constraints:
\begin{align*} 
\sum_{j}  \sum_{i} y_i ADJ\_REV_{i} & \ge 0.9\epsilon\\
y_i & > \frac{1}{M}\sum_{t} x_{it} \hspace{1cm} i \in \{1, \dots, n\}\\
c_t & \le C\\
\end{align*}

In [ ]:
# Define problem
model3 = plp.LpProblem("Attendance Maximization", plp.LpMaximize)

X = plp.LpVariable.dicts(name='X', indexs=(I,T), cat=plp.LpBinary)
Y = plp.LpVariable.dicts(name='Y', indexs=(I), cat=plp.LpBinary)
Z = plp.LpVariable.dicts(name='Z', indexs=(I))

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
#Maximize total number of students admitted

model3 += plp.lpSum(Y[i] for i in I)

In [ ]:


for i in I:
  model3 += M*Y[i] >= plp.lpSum(X[i][j] for j in T)
  model3 += Y[i] <= plp.lpSum(X[i][j] for j in T)
  model3 += Z[i] == plp.lpSum(X[i][j] for j in T) / sum([tw_dict[j][i] for j in range(3)])

  for j in T:
    model3 += X[i][j] <= tw_dict[j][i]

#we cannot force admitted students to leave
#if they intend to stay for the next time window

for i in I:
  model3 += X[i][1] >= X[i][0]*tw_dict[1][i]
  model3 += X[i][2] >= X[i][1]*tw_dict[2][i]

for j in T:
  model3 += plp.lpSum(X[i][j] for i in I) <= C

model3 += plp.lpSum(Z[i] * sample.iloc[i]["REVENUE"] for i in I) >= 0.9 * plp.value(model1.objective)

In [ ]:
model3.solve()

1

In [ ]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[model3.status])

total_visit = 0
for i in range(1500):
  for j in range(3):
    total_visit += X[i][j].varValue
print("Total visits: " + str(total_visit))

total_student_admits = 0
for i in range(1500):
  total_student_admits += Y[i].varValue
print("Total students admitted: " + str(total_student_admits))

rev = 0
for i in range(1500):
  rev += Z[i].varValue * sample.iloc[i]["REVENUE"]
print("Total revenue: " + str(rev))

Solution Status: Optimal
Total visits: 1441.0
Total students admitted: 979.0
Total revenue: 16526.5
